# QEK from A to Z

This notebook reproduces the results of the [QEK paper](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.107.042615).

At the end, you will be able to:
1. Extract the embeddings of a molecular dataset
2. Compile these embeddings into Pulse sequences for use on a Quantum Device or an amulator.
3. Run the Pulse sequences on a Quantum Device or an emulator.
4. Train an SVM with the QEK (Quantum Evolution Kernel) kernel and benchmark the performance reported in the paper.


In [ ]:
from dataclasses import dataclass

import numpy as np
import torch_geometric.datasets as pyg_dataset

## Dataset preparation

As in any machine learning task, we first need to load and prepare data. QEK can work with many types of graphs, including molecular graphs. For this tutorial, we will use the PTC-FM dataset, which contains such molecular graphs.

In [ ]:
# Load the original PTC-FM dataset
og_ptcfm = [data for data in pyg_dataset.TUDataset(root="dataset", name="PTC_FM")]

display("Loaded %s samples" % (len(og_ptcfm), ))

To extract machine-learning features from our dataset, we will need to configure a feature extractor. This library provides several feature extractors to either make use of a physical quantum device (QPU), or a variety of emulators.

To configure a feature extractor, we will need to give it a _compiler_, whose task is to take a list of graphs, extract embeddings and compile these embeddings to _sequences of pulses_, the format that can be executed  by either a QPU or an emulator. For this tutorial, our dataset is composed of molecule graphs, so we will use the `MoleculeGraphCompiler`:

In [ ]:
import qek.data.graphs as qek_graphs

compiler = qek_graphs.MoleculeGraphCompiler()

This library provides other compilers from other formats of graphs.

# Creating and executing a feature extractor from an emulator

The easiest way to process a graph is to compile and execute it for an emulator. QEK is built on top of Pulser, which provides several emulators. The simplest of these emulators is the `QutipEmulator`, which QEK uses for the `QutipExtractor`:

In [ ]:
import qek.data.extractors as qek_extractors

# Use the Qutip Extractor.
extractor = qek_extractors.QutipExtractor(
    # Once computing is complete, data will be saved in this file.
    path="saved_data.json",
    compiler=compiler
)

# Add the graphs using the compiler we've picked previously.
extractor.add_graphs(graphs=og_ptcfm)

# We may now compile them.
compiled = extractor.compile()
display("Compiled %s sequences" % (len(compiled), ))

As you can see, the number of sequences compiled is lower than the number of samples loaded. Some of this is due to limitations within the algorithm (not all graphs can be efficiently laid out for execution on a Quantum Device), while others are due to the limitations of the emulator we target (which at the time of this writing is limited to 50 qubits).

We may now run the extraction on the emulator:

In [ ]:
# Limit the number of qubits for this run, for performance reasons.
# You can increase this value to higher number of qubits, but this
# notebook will take longer to execute and may run out of memory.
max_qubits = 5
processed_dataset = await extractor.run(max_qubits=max_qubits) # Don't forget to `await`!
display("Extracted features from %s samples"% (len(processed_dataset), ))

If you wish to extract features from more samples, feel free to increase the value of `max_qubits` above. However, you will soon run into limitations of a quantum emulator, and possibly crash this notebook. At this point, you have other options, such as using `EmuMPSExtractor` instead of `QutipExtractor`, a more recent emulator that features much better performance in most cases, or you can run the extraction on a physical QPU.

# Creating and executing a feature extractor on a physical QPU

Once you have checked that low qubit sequences provide the results you expect on an emulator, you will generally want to move to a QPU.
For this, you will need either physical access to a QPU, or an account with [PASQAL Cloud](https://docs.pasqal.cloud), which provides
you remote access to QPUs built and hosted by Pasqal. In this section, we'll see how to use the latter.

If you don't have an account, just skip to the next section!

In [ ]:
HAVE_PASQAL_ACCOUNT = False # If you have a PASQAL Cloud account, fill in the details and set this to `True`.

if HAVE_PASQAL_ACCOUNT:
    processed_dataset = []

    # Use the QPU Extractor.
    extractor = qek_extractors.QPUExtractor(
        # Once computing is complete, data will be saved in this file.
        path="saved_data.json",
        compiler = compiler,
        project_id = "XXXX", # Replace this with your project id on the PASQAL Cloud
        username = "XXX",    # Replace this with your username on PASQAL Cloud
        password = None,     # Replace this with your password on PASQAL Cloud or enter it on the command-line
    )

    # Add the graphs, exactly as above.
    extractor.add_graphs(graphs=og_ptcfm)

    # We may now compile, exactly as above.
    compiled = extractor.compile()
    display("Compiled %s sequences" % (len(compiled), ))

    # Launch the execution.
    execution = extractor.run()
    display("Work enqueued with ids %s" % (extractor.batch_ids, ))

    # ...and wait for the results.
    processed_dataset = await execution
    display("Extracted features from %s samples"% (len(processed_dataset), ))

As you can see, the process is essentially identical to executing with an emulator. Note that, as of this
writing, the waiting line to access a QPU can be very long (typically several hours).

There are two main ways to deal with this:

1. `QPUExtractor` can be attached to an ongoing job from batch ids, so that you can resume your work
    e.g. after turning off your computer.
2. Pasqal CLOUD offers access to high-performance hardware-based emulators, with dramatically
    shorter waiting lines.

See [the documentation](https://pqs.pages.pasqal.com/quantum-evolution-kernel/) for more details.

## ...or using the provided dataset

For this notebook, instead of spending hours running the simulator on your computer, we're going to skip
this step and load on we're going to cheat and load the results, which are conveniently stored in `ptcfm_processed_dataset.json`.

In [ ]:
import qek.data.dataset as qek_dataset
processed_dataset = qek_dataset.load_dataset(file_path="ptcfm_processed_dataset.json")
print(f"Size of the quantum compatible dataset = {len(processed_dataset)}")

## A look at the results

We can check the sequence for one of the samples:

In [ ]:
dataset_example = processed_dataset[64]
dataset_example.draw_sequence()

In [ ]:
dataset_example.draw_register()

The results of executing the embedding on the Quantum Device are in field `state_dict`:

In [ ]:
display(dataset_example.state_dict)
print(f"Total number of samples: {sum(dataset_example.state_dict.values())}")

This dictionary represents an approximation of the quantum state of the device for this graph after completion of the algorithm.

- each of the keys represents one possible state for the register (which represents the graph), with each qubit (which represents a single node) being in state `0` or `1`;
- the corresponding value is the number of samples observed with this specific state of the register.

In this example, for instance, we can see that the state observed most frequently is `10000001010`, with 43/1000 samples.


## Machine learning-features

From the state dictionary, we derive as machine-learning feature the _distribution of excitation_. We'll use this in a second to define our kernel.

In [ ]:
dataset_example.draw_excitation()

# Train your first quantum machine learning algorithm

The next step is to use the results of the Quantum Device execution on a classical device (i.e. your computer) to create a Quantum Evolution Kernel. Since our algorithm combines steps that are executed on a Quantum Device and steps that are executed on a classical device, we call this a _hybrid algorithm_.

## Introducing the Quantum Evolution Kernel

For a graph $G$, let's call the excitation distribution $P_G$.

We may now construct the Quantum Evolution Kernel, or QEK. Mathematically, QEK defined as:
$$
K(G, G') = \exp \left( -\mu JS(P_G, P_{G'}) \right)
$$


where $\mu$ is an hyperparameter of our kernel and $JS$ is the Jensen-Shannon distance.

In [ ]:
from qek.kernel import QuantumEvolutionKernel as QEK
kernel = QEK(mu=2.)

Parameter $\mu$ controls the rate of exponential decay. A large value of $\mu$ makes QEK very sensitive to small variations of the Jensen-Shanon distance. Conversely, when $\mu$ is small, the kernel is less affected by small variations in of $JS$.

QEK compares two processed graphs by their distribution of excitations. If `a` and `b` are two graphs, a value of `kernel(a, b)` close to 1 indicates a big similarity between graphs `a` and `b`, while a value close to 0 means a small graph similarity.

Let's try that:

In [ ]:
graph_1 = processed_dataset[2]
graph_1.draw_register()
graph_1.draw_excitation()
display(f"Comparing a graph with itself: {kernel(graph_1=graph_1, graph_2=graph_1)}")

In [ ]:
graph_2 = processed_dataset[0]
graph_2.draw_register()
graph_2.draw_excitation()
display(f"Comparing two much non similar graphs: {kernel(graph_1=graph_1, graph_2=graph_2)}")

## Training and evaluation

In this part, we will calculate the kernel on the entire dataset of PTC-FM. We obtain an NxN matrix (where N is the number of graphs in the dataset). This matrix contains the similarities two by two of the graphs.

This precomputed kernel will allow us to evaluate the algorithm QEK. We will use an SVM (Support Vector Machine) to learn how to predict the toxicity of a molecule based on the precomputed kernel. This task is handled with the `train_and_evaluate_ml_model`.

In [ ]:
train_kernel = kernel.create_train_kernel_matrix(processed_dataset)
y_tot = [data.target for data in processed_dataset]

In [ ]:
import sklearn.svm as svm
from sklearn.metrics import balanced_accuracy_score, f1_score, make_scorer
from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold


@dataclass
class MLResults:
    """
    Stores the results of a machine learning model evaluation.
    
    This class provides attributes to store the mean and standard deviation of F1 score 
    and balanced accuracy, which are common metrics used in classification problems.

    Attributes:
        f1_score (float): Mean F1 score of the model.
        std_f1_score (float): Standard deviation of F1 scores across different folds.
        balanced_acc (float): Mean balanced accuracy of the model.
        std_balanced_acc (float): Standard deviation of balanced accuracies across different folds.
    """
    f1_score: float
    std_f1_score: float
    balanced_acc: float
    std_balanced_acc: float


def train_and_evaluate_ml_model(K: np.ndarray, targets: list[int], seed1: int|None = None,
                                seed2: int|None = None):
    """
    Trains and evaluates a Support Vector Machine (SVM) model on the provided kernel.

    It employs a stratified k-fold cross-validation strategy with repeated splits
    to ensure robustness and reliability of the results.

    Parameters:
        K (np.ndarray): The precomputed kernel matrix.
        targets (list[int]): A list of target values for the training data.
        seed1 (int, optional): The random state used for stratified k-fold cross-validation.
            Defaults to None.
        seed2 (int, optional): The random state used for the SVM estimator. Defaults to None.

    Returns:
        MLResults: An instance containing the mean and standard deviation of F1 score and
            balanced accuracy.
    """
    C_list = np.linspace(0.001, 100, 100)
    param_grid = {"C": C_list}
    scoring = {"balanced_accuracy": make_scorer(balanced_accuracy_score),
                "f1_score": make_scorer(f1_score, average="weighted")
                }

    skf = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=seed1)


    estimator = svm.SVC(kernel="precomputed", random_state=seed2)
    grid_search = GridSearchCV(estimator, param_grid, scoring=scoring,
                                    cv=skf, refit=False, n_jobs=8)
    

    result = grid_search.fit(K, targets).cv_results_
    max_f1_score = np.mean(result["mean_test_f1_score"]) 
    final_f1_std = np.mean(result["std_test_f1_score"])
    max_bal_acc = np.mean(result["mean_test_balanced_accuracy"]) 
    std_bal_acc = np.mean(result["std_test_balanced_accuracy"])
    final_score = MLResults(f1_score=max_f1_score, std_f1_score=final_f1_std,
                            balanced_acc=max_bal_acc, std_balanced_acc=std_bal_acc)
    return final_score

In [ ]:
results = train_and_evaluate_ml_model(K=train_kernel, targets=y_tot, seed1=42, seed2=42)

### Results

We are using two metrics:
- The F1 score is a way to measure how well a model performs, especially when the data is uneven (e.g., more examples of one category than another). It combines two important aspects: how precise the model is (how many of the predicted positives are actually positive) and how well it captures all the actual positives (recall). It provides a single number that balances these two aspects, making it useful for evaluating performance in real-world scenarios where some categories are much more common than others.

- Balanced accuracy is a method to evaluate a model's performance fairly, even when the data is imbalanced (e.g., one category is much more frequent than others). Instead of just looking at overall accuracy, which can be misleading in such cases, balanced accuracy considers how well the model performs for each category separately and then averages these performances. This ensures that the evaluation is not skewed by the more common categories, giving a more honest picture of the model's effectiveness across all categories.


Inside the `train_and_evaluate_ml_model`function, we split our data multiple times to ensure each fold is representative of the overall class distribution, helping to mitigate bias. The mean value and standard deviation (std) of each metrics from this process provide an average performance measure and the variability of that performance across different data splits, giving you a robust understanding of your model's consistency and reliability.


In [ ]:
print(f"Mean F1 score = {results.f1_score}")
print(f"Standard deviation of F1 score = {results.std_f1_score}")
print(f"Mean balanced accuracy {results.balanced_acc}")
print(f"Standard deviation of balanced accuracy {results.std_balanced_acc}")